In [1]:
"""
This is taking forever, besides... do I really just want a raster with mk.slope? I want some sort of seasonal decomposition of ppt trends by month (June, July, Aug).

This would go a long way to reduce the size of the computations as well. But the lat/lon values are weird in this dataset as it's been reformated for cesm atm forcing.
"""

import os
import xarray as xr
import dask
import pymannkendall as mk
from dask.diagnostics import ProgressBar

## Pathing ### ----------------
# nldas_dir = "/glade/campaign/cgd/tss/people/swensosc/"
cesm_forcing_dir = "/glade/campaign/cesm/cesmdata/inputdata/"
nldas_dir = os.path.join(cesm_forcing_dir,
                         "atm/datm7/atm_forcing.datm7.NLDAS2.0.125d.v1")
print(f' NLDAS ATM forcing data dir contents:\n {os.listdir(nldas_dir)}')

ppt_dir = os.path.join(nldas_dir, "Precip")
ppt_fns = [os.path.join(ppt_dir, fn) for fn in os.listdir(ppt_dir)]


 NLDAS ATM forcing data dir contents:
 ['Solar', 'ctsmforc.NLDAS2.cdf5.0.125d.v1.ESMFmesh_120620_c210330.nc', 'Precip', 'README_190425', 'TPQWL']


In [2]:
# Open all files and preprocess nldas data to optimize memory
from functools import partial

# Function to preprocess each dataset by selecting a region and variable
def _preprocess(ds, lon_bnds, lat_bnds):
    ds = ds.sel(lon=slice(*lon_bnds), lat=slice(*lat_bnds))
    return ds[['PRECTmms']]  # Only keep the 'PRECTmms' variable

# Partially define function
lon_bnds, lat_bnds = (100, 250), (75, 150)
partial_func = partial(_preprocess, lon_bnds=lon_bnds, lat_bnds=lat_bnds)

print('Initializing dataset (lazy loading)...')
ds = xr.open_mfdataset(
    ppt_fns, preprocess=partial_func, data_vars='minimal'
).chunk({'time': -1, 'lat': 5, 'lon': 5})
print('Finished lazy loading dataset!')


Initializing dataset (lazy loading)...
Finished lazy loading dataset!


In [ ]:

# ds = xr.open_mfdataset(ppt_fns, combine='by_coords')
# ds = ds.chunk({'time': -1, 'lat': 5, 'lon': 5})
# print(ds['PRECTmms'])

# Function to apply Mann-Kendall test pixel by pixel
def mann_kendall_trend(x):
    result = mk.original_test(x)
    return result.slope  # Return the trend slope

trend_mk = xr.apply_ufunc(
    mann_kendall_trend, 
    ds['PRECTmms'], 
    vectorize=True, 
    input_core_dims=[['time']],
    dask='parallelized',
    output_dtypes=[float]
)

print('Computing MK...')
with ProgressBar():
    trend_mk_computed = trend_mk.compute()
print('Done!') 
trend_mk_computed.to_netcdf('nldas_mk_trend_output.nc')
print('Saved MK .nc file to nldas_mk_trend_output.nc') 

Computing MK...
[##################################      ] | 85% Completed | 32m 39ss


In [ ]:
print(os.getcwd())